<a href="https://colab.research.google.com/github/emil2397/useful_tools/blob/main/PSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PSI index

Ноутбук запускается по сегодняшней дате

In [ ]:
# требуется utils

import sys
sys.path.append("..")

In [ ]:
def load_large_data_to_pd2(spark, training_mode=False, base_path= '', nbatch = 400, core_feats = ['SupplierID', 'date'], load_feats = None):

        start_time0 = time.time()

        if training_mode:
                sdf = spark.read.parquet(base_path).where(f"...")
        else:
                sdf = spark.read.parquet(base_path)

        all_cols = sdf.columns

        if(load_feats is not None):
                all_cols = load_feats

        tot_batch = len(all_cols)//nbatch + 1
        res = []
        start_time = time.time()
        df = sdf.select(all_cols[:nbatch]).toPandas().sort_values(core_feats).reset_index(drop=True)
        res.append(df)
        print('join: step', 1, '/', tot_batch, df.shape, 'time: {0}s'.format(round( (time.time() - start_time), 1)))

        for i in range(1,tot_batch):
                gc.collect()
                start_time = time.time()
                df = sdf.select(core_feats + all_cols[i*nbatch:(i+1)*nbatch]).toPandas().sort_values(core_feats).reset_index(drop=True)
                res.append(df.drop(core_feats, axis=1))
                print('join: step', i+1, '/', tot_batch, df.shape, 'time: {0}s'.format(round( (time.time() - start_time), 1)))
                del df

        start_time = time.time()
        df = pd.concat(res, axis=1)
        del res
        gc.collect()
        print('concat', df.shape, 'time: {0}s'.format(round( (time.time() - start_time), 1)))
        print('total time: {0}s'.format(round( (time.time() - start_time0), 1)))


        return df

def load_data(base_path, nbatch, dt1='2019-11-01', training_mode=False, base_features=None):

    core_feats = [... ]

    df = load_large_data_to_pd2(spark, training_mode, '...' + base_path +"/...", nbatch ,
                              load_feats=base_features+core_feats
                              )

    gc.collect()
    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['date'] >= dt1)].copy()
    df = df.sort_values(['date', 'SupplierID']).reset_index(drop=True)
    df['ID'] = df.index.copy()
    df['w'] = np.log(df['...'])
    gc.collect()


    return df

In [ ]:
def predict_new_data_ts(X, path):
        
        if path:
            import subprocess
            
            m_f = os.path.join(os.getcwd(), 'models')
            modelfolder_local = os.path.join(m_f, f"{path.split('/')[-1]}")
            
            if not os.path.exists(m_f):
                os.mkdir(m_f)
            
            if not os.path.exists(modelfolder_local):
                os.mkdir(modelfolder_local)
            
            seed_date = lambda x: [x.split('/')[-1].split('_')[0], '-'.join([x.split('/')[-1].split('_')[-3], x.split('/')[-1].split('_')[-1][:2], x.split('/')[-1].split('_')[-2]])]
            
            if not len(os.listdir(modelfolder_local))>0:

                subprocess.run(['hdfs', 'dfs', '-get', '-f', path+'/*', modelfolder_local], capture_output=True,
                    text=True).stdout
            
            model_names = [model_name for model_name in os.listdir(modelfolder_local)]

            model_files = [os.path.join(modelfolder_local, f) for f in model_names if '.model' in f]
            
            s = sorted([seed_date(x) for x in model_files])
            
            model_files = sorted(model_files, key= lambda x:seed_date(x))
            
            max_date = s[-1][1]
            min_date = s[0][1]
            model_files = sorted(model_files, key= lambda x:seed_date(x))
            tot_models = len(set([x[0] for x in s]))
            
            if tot_models >0:
                    
                    predict = np.zeros(shape=(X.shape[0],))
                    features = list(pd.read_csv(os.path.join(modelfolder_local, 'train_features.txt')).columns)
                    
                    for i in range(len(model_files)):
                        
                            curr_seed, curr_date = s[i]
                            
                            model_loaded = lightgbm.Booster(model_file=model_files[i])
                            
                            
                            if(curr_date!=max_date):
                                    curr_ind = np.where(X['date'] == curr_date)[0]
                            else:
                                    curr_ind = np.where(X['date'] >= curr_date)[0]
                                    
                            if(X.iloc[curr_ind][features].shape[0]!=0):                     
                                    predict[curr_ind] += model_loaded.predict(X.iloc[curr_ind][features].round(6)) / tot_models
                                    
                    null_index = np.where(X['date'] < min_date)
                    predict[null_index] = -100
                    
                    return np.round(predict, decimals = 5)
            else:
                print('Models not found!!!')

In [ ]:
def calculate_base_values(all_pds,sc):

    def calc_bin_stats(df0, sc, npart=10 ):
            df0.loc[:,'bin'],bins = pd.qcut(df0[sc],npart,duplicates='drop',retbins=True)
            df = df0[[sc,'bin']].copy()
            df['bin'] = df['bin'].apply(lambda x: x.left)
            df['bin'] = df['bin'].astype(str)
            df = df['bin'].value_counts().reset_index().rename({'index':'bin','bin':'counter_base'},axis=1)
            df['base_prc'] = df['counter_base'] / sum(df['counter_base'])
  

            return df,bins
        
    def psi(new, base):
        return round(sum((new-base) * np.log(new/base)),3)


    score_names = []
    bins_buckets = []
    bins_used = []
    base_prcs = []

    base,bins = calc_bin_stats(all_pds.copy(), sc)

    score_names.append(sc)
    bins_buckets.append(list(base['bin'].values))
    base_prcs.append(list(base['base_prc'].values))
    bins_used.append(list(bins))

    params = tuple([list(base['bin'].values), list(base['base_prc'].values), list(bins_used[0])])
    
    return params

In [ ]:
spark = run_spark(...)

# Расчет словарей pd/feats


In [ ]:
# ! pip install pydoop

In [ ]:
def calculate_base_stats(model_folder,models_names,df_name, base_start_date, base_end_date):

    feats_all = []

    for model in models_names:
        with hd.open(f"{model_folder}/{model}/train_features.txt") as f:
            feats_all=feats_all+ list(pd.read_csv(f).columns)

    feats=list(set(feats_all))
    
    df = load_data(df_name, 500, base_start_date, training_mode=True, base_features=feats)
    
    print("1 LOAD OK!")
    
    for model in tqdm(models_names):
        df[model] = predict_new_data_ts(df, model_folder+f'/{model}')

    df['pd'] = df[models_names].mean(axis=1)
    
    print("2 PD OK!")
    
    base_mask = (df["date"] >= base_start_date) & (df["date"] <= base_end_date) & (df["..."])
    base_bins_feats = {}
    feats_extended = ["pd"] + feats

    for col in feats_extended:
        base_bins_feats[col] = calculate_base_values(df[base_mask], sc=col)
    
    print("3 DICTS OK!")
    
    return df, base_bins_feats

In [ ]:
if (0):
    df_gen, base_bins_feats = calculate_base_stats(model_folder,models_names,df_name,base_start_date, base_end_date)
    
    with open('config.json', 'wb') as fp:
        pickle.dump(base_bins_feats, fp)

join: step 1 / 1 (146188, 148) time: 34.9s
concat (146188, 148) time: 0.3s
total time: 69.4s


/home/eyusifov@ozon/python3_env/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


1 LOAD OK!


100%|██████████| 4/4 [06:24<00:00, 96.22s/it]


2 PD OK!
3 DICTS OK!


In [1]:
# with open('config.json', 'rb') as fp:
#     base_bins_feats = pickle.load(fp)

# print(base_bins_feats)
# print(type(base_bins_feats))

## Расчет тестовых

In [ ]:
def test_psi(spark, base_bins_feats, test_path_read, only_critical=False, part_filter='', day=None, logger=print):
    
    def calculate_psi_index(params,  feat, df_test=None, npart=10):

        def psi(new, base):
            return round(sum((new-base) * np.log(new/base)),3)

        bin_df,base_prc,bins = params
        base_res = pd.DataFrame({'bin':bin_df,'base_prc':base_prc})
        df = df_test.copy()   
        df.loc[:,'bin'] = pd.cut(df[feat],bins=bins,include_lowest=True)
        upper_bounds = {}

        for i,lower_bound in enumerate(bins):
            if i!=len(bins)-1:
                upper_bounds[round(lower_bound,2)] = round(bins[i+1],4)
            else:
                upper_bounds[round(lower_bound,2)] = 10000.

        df.loc[:,'bin_upper'] = df['bin'].apply(lambda x: x.right)
        df.loc[:,'bin_upper'] = df['bin_upper'].astype(str)

        df.loc[:,'bin'] = df['bin'].apply(lambda x: x.left)
        df.loc[:,'bin'] = df['bin'].astype(str)


        df_res = df['bin'].value_counts().reset_index().rename({'index':'bin','bin':'counter_current'},axis=1).copy()
        df_res['bin_prc'] = df_res['counter_current'] / sum(df_res['counter_current'])
        base_res['bin'] = base_res['bin'].astype(str)    
        res = pd.merge(df_res,base_res,on='bin',how='outer')
        ub = df[['bin','bin_upper']].groupby('bin').first().reset_index()
        res = pd.merge(res,ub,on='bin',how='inner')
        res.drop('bin',axis=1,inplace=True)
        res.rename({'bin_upper':'bin'},axis=1,inplace=True)
        res = res.sort_values(by=['bin'],ascending=False,axis=0,na_position='first')
        res['bin'] = res['bin'].astype(float).round(2)

        return psi(res['bin_prc'],res['base_prc']),res
    

    test_path_read = test_path_read.format(day.strftime('%Y%m%d'))

    feats_load = list(base_bins_feats.keys())

    print(f"DATE of scoring: {datetime.now().strftime('%Y-%m-%d')}")
    
        
    print(f"DATE of file: {day.strftime('%Y-%m-%d')}")

    if len(part_filter)==0:
        df_test = spark.read.parquet(test_path_read).toPandas()[feats_load]
    else:
        df_test = spark.read.parquet(test_path_read).where(f"{part_filter}").toPandas()[feats_load]
    
    output = {}
    
    for feat in base_bins_feats.keys():
        output[feat] = calculate_psi_index(params=base_bins_feats[feat], feat=feat, df_test=df_test)[0]
    
    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df = output_df.rename(columns={0:'psi_value'})
    output_df["psi_value"] = output_df["psi_value"].round(4)    


    if (output_df['psi_value'] > 0.25).any() & only_critical:
        output_df=output_df[output_df['psi_value'] > 0.25]
        logger(f"YES CRITICAL PSI: \n {msg}")
        
    else:
        if (output_df['psi_value'] > 0.25).any():
            logger(f"YES CRITICAL PSI")
        else:
            logger(f"NO CRITICAL PSI")
        
    
    return output_df

In [ ]:
config = {
    'sellers' : {"base_bins_feats": base_bins_feats, 
                "test_path_read": '...',
                "only_critical": False,
                "part_filter": "..."
                },

}

In [ ]:
import datetime as DT
today = DT.date.today()
week_ago = today - DT.timedelta(days=7)

dates = {}

for day in pd.date_range(week_ago,today):
    
    dates[day.strftime("%Y-%m-%d")] = test_psi(spark=spark, day=day, **config["sellers"]).to_dict()["psi_value"]
                                                
    print("\n")
    
result=pd.DataFrame.from_dict(dates, orient='columns')
result=result.sort_values(by=result.columns[-1],ascending=False)